In [1]:
#### Grabbing info from TJ's model explorations

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from statsmodels.formula.api import ols

df = pd.read_csv('data/kc_house_data.csv')

In [3]:
#This should be all data cleaning and pre-processimg
grade_num = []
for index, row in df.iterrows():
    grade_num.append(int((str(row['grade'])[0:2])))
df['grade_num'] = grade_num
df.drop('grade', axis = 1)
df['sqft_living_ratio'] = df['sqft_living'] / df['sqft_living15']
df['sqft_living_ratio'] = df['sqft_living_ratio'].loc[(np.abs(stats.zscore(df.sqft_living_ratio)) < 2)]
df = df.dropna(subset = ['sqft_living_ratio'])
df['sqft_lot_ratio'] = df['sqft_lot'] / df['sqft_lot15']
df['sqft_lot_ratio'] = df['sqft_lot_ratio'].loc[(np.abs(stats.zscore(df.sqft_lot_ratio)) < 2)]
df = df.dropna(subset = ['sqft_lot_ratio'])
df.waterfront.fillna(value='NO', inplace = True)
df.waterfront.replace( to_replace = ['NO','YES'], value =  [0,1], inplace = True)
df = df[df['bedrooms'] < 33]
df['sqft_basement'] = df['sqft_basement'].replace({'?': 0.0})
df['sqft_basement'] = df['sqft_basement'].astype(float)
df['view'] = df['view'].replace({'NONE': 0, 'AVERAGE': 1, 'GOOD': 2, 'FAIR': 3, 'EXCELLENT':4})
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
ohe.fit_transform(df[['zipcode']])
ord_enc = OrdinalEncoder()
ord_enc.fit_transform(df[['bedrooms','bathrooms','floors','grade_num']])



array([[ 2.,  2.,  0.,  3.],
       [ 2.,  7.,  2.,  3.],
       [ 3., 10.,  0.,  3.],
       ...,
       [ 1.,  1.,  2.,  3.],
       [ 2.,  8.,  2.,  4.],
       [ 1.,  1.,  2.,  3.]])

In [4]:
from sklearn.model_selection import train_test_split
X = df[['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view',
       'sqft_above', 'sqft_basement', 'zipcode','sqft_living15', 'sqft_lot15', 'grade_num']]


X_train, X_test, y_train, y_test = train_test_split(X,df.price)

training_data = pd.concat([X_train,y_train], axis = 1)
testing_data = pd.concat([X_test,y_test], axis = 1)

In [5]:
from sklearn.linear_model import LinearRegression

baseline_model = LinearRegression()

In [6]:
from sklearn.model_selection import cross_validate, ShuffleSplit

most_correlated_feature = 'sqft_living'

splitter = ShuffleSplit(n_splits=3, test_size=0.25, random_state=0)

baseline_scores = cross_validate(
    estimator=baseline_model,
    X=X_train[[most_correlated_feature]],
    y=y_train,
    return_train_score=True,
    cv=splitter
)

print("Train score:     ", baseline_scores["train_score"].mean())
print("Validation score:", baseline_scores["test_score"].mean())

Train score:      0.46703818504446115
Validation score: 0.47794820895447865


In [7]:
ols('y_train ~ X_train', data = training_data).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                y_train   R-squared:                       0.600
Model:                            OLS   Adj. R-squared:                  0.599
Method:                 Least Squares   F-statistic:                     1754.
Date:                Tue, 16 Nov 2021   Prob (F-statistic):               0.00
Time:                        14:22:30   Log-Likelihood:            -2.0855e+05
No. Observations:               15244   AIC:                         4.171e+05
Df Residuals:                   15230   BIC:                         4.172e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -4.464e+07   3.42e+06    -13.058      0.000   -5.13e+07   -3.79e+07
X_train[0]    -2.8e+04   2544.404    -11.004      0.000    -3.3e+04    -2.3e+04
X_train[1]  -1.889e+04   3956.699     -4.775      0.000   -2.66e+04   -1.11e+04
X_train[2]    142.1348     23.521      6.043      0.000      96.030     188.239
X_train[3]      0.0038      0.118      0.033      0.974      -0.227       0.235
X_train[4]  -9070.2909   4396.394     -2.063      0.039   -1.77e+04    -452.833
X_train[5]   5.349e+05    2.4e+04     22.308      0.000    4.88e+05    5.82e+05
X_train[6]   7.092e+04   3020.085     23.482      0.000     6.5e+04    7.68e+04
X_train[7]      1.4421     23.383      0.062      0.951     -44.391      47.275
X_train[8]     51.3394     23.232      2.210      0.027       5.802      96.877
X_train[9]    450.0594     34.839     12.918      0.000     381.770     518.349
X_train[10]    55.9156      5.045     11.083      0.000      46.027      65.804
X_train[11]    -0.5610      0.137     -4.089      0.000      -0.830      -0.292
X_train[12]  9.738e+04   2674.899     36.405      0.000    9.21e+04    1.03e+05
==============================================================================
Omnibus:                     7259.640   Durbin-Watson:                   1.987
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            95398.004
Skew:                           1.953   Prob(JB):                         0.00
Kurtosis:                      14.616   Cond. No.                     2.00e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large,  2e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [8]:
ziplist = ohe.categories_[0].tolist()
zipliststr = []
for i in ziplist: 
    zipliststr.append(str(i))

ohe = OneHotEncoder(sparse = False)
ohe.fit_transform(df[['zipcode']])
zip_encoded = ohe.fit_transform(df[['zipcode']])
zip_enc_df = pd.DataFrame(zip_encoded, columns=ohe.categories_[0], index=df.index)


df.drop("zipcode", axis=1, inplace=True)
df = pd.concat([df, zip_enc_df], axis=1)

In [9]:
ohegrade = OneHotEncoder(sparse = False)
ohegrade.fit_transform(df[['grade']])
grade_encoded = ohe.fit_transform(df[['grade']])
grade_enc_df = pd.DataFrame(grade_encoded, columns=ohegrade.categories_[0], index=df.index)


df.drop("grade", axis=1, inplace=True)
df = pd.concat([df, grade_enc_df], axis=1)


gradelist = ohegrade.categories_[0].tolist()
gradeliststr = []
for i in gradelist: 
    gradeliststr.append(str(i))

#ord_enc = OrdinalEncoder()
#ord_enc.fit_transform(df[['bedrooms','bathrooms','floors','grade_num']])


In [10]:
df


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,10 Very Good,11 Excellent,12 Luxury,13 Mansion,4 Low,5 Fair,6 Low Average,7 Average,8 Good,9 Better
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,7237550310,5/12/2014,1230000.0,4,4.50,5420,101930,1.0,0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,5/21/2014,360000.0,3,2.50,1530,1131,3.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
21593,6600060120,2/23/2015,400000.0,4,2.50,2310,5813,2.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
21594,1523300141,6/23/2014,402101.0,2,0.75,1020,1350,2.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
21595,291310100,1/16/2015,400000.0,3,2.50,1600,2388,2.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
Seattle_zipcodes = [98101, 98102, 98103, 98104, 98105, 
                    98106, 98107, 98108, 98109, 98112, 
                    98115, 98116, 98117, 98118, 98119, 
                    98121, 98122, 98125, 98126, 98133, 
                    98134, 98136, 98144, 98146, 98154, 
                    98164, 98174, 98177, 98178, 98195, 
                    98199 ]

df_Seattle = df[df['zipcode'].isin(Seattle_zipcodes)]
df_Outside = df[~df['zipcode'].isin(Seattle_zipcodes)]
df_original = df

KeyError: 'zipcode'

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse = False)
ohe.fit_transform(df[['zipcode']])
zip_encoded = ohe.fit_transform(df[['zipcode']])
zip_enc_df = pd.DataFrame(zip_encoded, columns=ohe.categories_[0], index=df.index)
df = pd.concat([df, zip_enc_df], axis=1)
ziplist = ohe.categories_[0].tolist()
zipliststr = []
for i in ziplist: 
    zipliststr.append(str(i))
zipliststr
ord_enc = OrdinalEncoder()
ord_enc.fit_transform(df[['bedrooms','bathrooms','floors','grade_num']])

In [ ]:
X = df.drop(columns = ['id','date','price','lat','long','sqft_living_ratio','sqft_lot_ratio','condition','yr_built','yr_renovated','grade','zipcode'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,df.price)

training_data = pd.concat([X_train,y_train], axis = 1)
testing_data = pd.concat([X_test,y_test], axis = 1)

In [ ]:
from sklearn.linear_model import LinearRegression

baseline_model = LinearRegression()

In [ ]:
from sklearn.model_selection import cross_validate, ShuffleSplit

splitter = ShuffleSplit(n_splits=3, test_size=0.25, random_state=0)

baseline_scores = cross_validate(
    estimator=baseline_model,
    X=training_data[['sqft_living']],
    y=training_data.price,
    return_train_score=True,
    cv=splitter
)

print("Baseline Train score:     ", baseline_scores["train_score"].mean())
print("Baseline Validation score:", baseline_scores["test_score"].mean())


In [ ]:
second_model = LinearRegression()

second_model_scores = cross_validate(
    estimator=second_model,
    X=training_data.drop(columns = 'price'),
    y=y_train,
    return_train_score=True,
    cv=splitter
)

print("Second Model Train score:     ", second_model_scores["train_score"].mean())
print("Second Model Validation score:", second_model_scores["test_score"].mean())


In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import StandardScaler

X_train_for_RFECV = StandardScaler().fit_transform(X_train)
model_for_RFECV = LinearRegression()

# Instantiate and fit the selector
selector = RFECV(model_for_RFECV, cv=splitter)
selector.fit(X_train_for_RFECV, y_train)

# Print the results
print("Was the column selected?")
for index, col in enumerate(X_train.columns):
    print(f"{col}: {selector.support_[index]}")

In [ ]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train,y_train)
print(linreg.score(X_train, y_train))
print(linreg.score(X_test, y_test))
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

from sklearn.metrics import mean_squared_error

train_residuals = y_hat_train - y_train
test_residuals = y_hat_test - y_test

# Calculate training and test MSE

train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = mean_squared_error(y_test, y_hat_test)
print('Train Root Mean Squarred Error:', np.sqrt(train_mse))
print('Test Root Mean Squarred Error:', np.sqrt(test_mse))